<a href="https://colab.research.google.com/github/tadoran/mv_prices/blob/master/MVideo_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Вставить внутрь кавычек артикулы, по 1 на строку
vibs_txt = """
20020715
20056945
20056876
20059186
20036721
20032232
20035243
20030060
20056128
20059172
20039870
20041365
20059642
20061250
20062758
20059569
20036699
20029414
20035264
20038766
20038850
20028333
20059570
20055932
20056944
20035244
20030059
20042739
20041364

"""
VIB_ARTICLES = vibs_txt.strip().split("\n")

In [0]:
import requests as r
import pandas as pd
from datetime import datetime
from google.colab import files

In [0]:
def send_mv_request(query_dict):
    headers = ({
            "origin": "https://www.mvideo.ru",
            "content-type": "application/json"
            })
    responce = r.post(url="https://www.mvideo.ru/.rest/graphql", json=query_dict, headers=headers)
    return responce.json()

In [0]:
# Цены
price_request_Q = """
query prices($productIds: [String]!, $regionId: String!) {
  prices(productIds: $productIds, regionId: $regionId) {
    productId
    actionPrice
    basePrice
    economy
  }
}
"""

# Акции
labels_request_Q = """
    query labels($productIds: [String]!, $parameters: BaseQueryParameters) {
        labels(productIds: $productIds, parameters: $parameters) {
            productId
            promotionLabels {
                name
            }
        }
    }
    """

# Описания VIB'ов
devices_query_Q = """
query products($productIds: [String]!) {
  products(productIds: $productIds) {
    productId
    name {
      cyrillic
      latin
    }
    brand {
      name
    }
  }
}
"""



In [0]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [0]:
chunk_size = 100
failed_chunks = []
results_chunks = []
VIB_ARTICLES_chunks = [VIB_ARTICLES_chunk for VIB_ARTICLES_chunk in chunks(VIB_ARTICLES, chunk_size)]


for VIB_ARTICLES_chunk in VIB_ARTICLES_chunks:
    
    price_request  = {"operationName":"prices",  "variables": {"productIds": VIB_ARTICLES_chunk, "regionId":"1"},   "query": price_request_Q }
    labels_request = {"operationName":"labels",  "variables": {"productIds": VIB_ARTICLES_chunk},                   "query": labels_request_Q}
    devices_query  = {"operationName":"products","variables": {"productIds": VIB_ARTICLES_chunk},                   "query": devices_query_Q }

    try:
        # Prices
        prices_df = pd.DataFrame(send_mv_request(price_request)["data"]["prices"]).set_index("productId")

        # Labels
        labels = send_mv_request(labels_request)
        labels_list = []
        for el in labels["data"]["labels"]:
            for el2 in el["promotionLabels"]:
                labels_list.append([el["productId"], el2["name"]])
        labels_df = pd.DataFrame(labels_list,columns=["Article", "Label"])
        labels_df2 = pd.get_dummies(labels_df, columns=["Label"], prefix="", prefix_sep="").groupby("Article").sum()

        # Devices
        devices = send_mv_request(devices_query)
        devices_df = pd.DataFrame(
            [[el['brand']['name'], el['name']['cyrillic'], el['name']['latin'],el['productId']] for el in devices['data']['products']],
            columns=["Brand","DesrRU","DesrEN","Article"]
            ).set_index("Article")

        results_chunks.append(devices_df.join(prices_df.astype(int), how="outer").join(labels_df2.astype(int), how="outer"))
    except:
        failed_chunks.append(VIB_ARTICLES_chunk)
        if chunk_size > 1:
            chunk_size = chunk_size // 2
            VIB_ARTICLES_chunks += chunks(VIB_ARTICLES_chunk, chunk_size)

    
results = pd.concat(results_chunks,sort=False).fillna(0)
cur_datetime_str = datetime.today().strftime('%d.%m.%Y__%H-%M')

In [107]:
results

,Brand,DesrRU,DesrEN,actionPrice,basePrice,economy,Premium,«Рассрочка 0%» на всё,Двойной кэшбэк,Кэшбэк 10%,Успейте купить
20029414,Bosch,Пылесос ручной (handstick) Bosch BCH6ATH25K,pylesos-ruchnoi-handstick-bosch-bch6ath25k-200...,15590,15590,0,0.0,0.0,1,0.0,0.0
20035264,Bosch,Пылесос с контейнером для пыли Bosch BGS2UPWER3,pylesos-s-konteinerom-dlya-pyli-bosch-bgs2upwe...,14990,14990,0,0.0,1.0,1,1.0,0.0
20036699,Bosch,Пылесос ручной (handstick) Bosch BCH6256N1,pylesos-ruchnoi-handstick-bosch-bch6256n1-2003...,15990,26990,11000,0.0,0.0,1,1.0,1.0
20038766,Bosch,Пылесос с контейнером для пыли Bosch Relaxx'x ...,pylesos-s-konteinerom-dlya-pyli-bosch-relaxx-x...,23190,23190,0,0.0,1.0,1,1.0,0.0
20039870,Bosch,Пылесос с пылесборником Bosch Ergomaxx'x (Carp...,pylesos-s-pylesbornikom-bosch-ergomaxx-x-carpe...,16990,24990,8000,1.0,1.0,1,1.0,1.0
20041365,Bosch,Пылесос с пылесборником Bosch Cosyy'y ProFamil...,pylesos-s-pylesbornikom-bosch-cosyy-y-profamil...,15990,15990,0,0.0,1.0,1,1.0,0.0
20059569,Bosch,Пылесос ручной (handstick) Bosch Flexxo Serie ...,pylesos-ruchnoi-handstick-bosch-flexxo-serie-4...,15990,15990,0,0.0,1.0,1,1.0,0.0
20059642,Bosch,Пылесос с пылесборником Bosch GL-20,pylesos-s-pylesbornikom-bosch-gl-20-20059642,4490,7990,3500,0.0,0.0,1,0.0,0.0
20061250,Bosch,Пылесос с пылесборником Bosch GL-30 BSGL3MULT2,pylesos-s-pylesbornikom-bosch-gl-30-bsgl3mult2...,3490,3490,0,0.0,0.0,1,0.0,0.0
20062758,Bosch,Робот-пылесос Bosch Roxxter Serie | 6 BCR1ACG,robot-pylesos-bosch-roxxter-serie-6-bcr1acg-20...,69990,84990,15000,0.0,0.0,1,1.0,0.0


In [0]:
filename = 'mv_' + cur_datetime_str + '.csv'
with open(filename, 'w') as f:
    # https://stackoverflow.com/a/18239236
    f.write('\ufeff' + results.to_csv(sep=";", decimal=","))

files.download(filename)

In [0]:
# prices_df = pd.DataFrame(send_mv_request(price_request)["data"]["prices"]).set_index("productId")
# labels = send_mv_request(labels_request)
# labels_list = []
# for el in labels["data"]["labels"]:
#     for el2 in el["promotionLabels"]:
#         labels_list.append([el["productId"], el2["name"]])

# labels_df = pd.DataFrame(labels_list,columns=["Article", "Label"])
# labels_df2 = pd.get_dummies(labels_df, columns=["Label"], prefix="", prefix_sep="").groupby("Article").sum()

# devices = send_mv_request(devices_query)
# devices_df = pd.DataFrame(
#     [[el['brand']['name'], el['name']['cyrillic'], el['name']['latin'],el['productId']] for el in devices['data']['products']],
#     columns=["Brand","DesrRU","DesrEN","Article"]
#     ).set_index("Article")

# cur_datetime_str = datetime.today().strftime('%d.%m.%Y__%H-%M')
# results = devices_df.join(prices_df, how="outer").join(labels_df2, how="outer")